In [12]:
import os
import pandas as pd
import numpy as np
import json
import sys

from matplotlib import pyplot as plt
from pandas.io.json import json_normalize

import logging
logger = logging.getLogger(__name__)

# These lines allow me to see logging.info messages in my jupyter cell output
logger.addHandler(logging.StreamHandler(stream=sys.stdout)) 
logger.setLevel(logging.DEBUG)

# Grab subject & trial info

In [13]:
class subject_data():

    def __init__(self, subject_data_folder):
        
        self.sub_id = os.path.split(subject_data_folder)[-1]
        self.data_parent_folder = os.path.join(*os.path.split(subject_data_folder)[:-1])
        self.data_folder = subject_data_folder
        
        self.experiment_settings = None # Making the attribute known
        self.read_experiment_settings_from_file() # self.experiment_settings is now set
        #self.experiment_settings['figure_out_folder'] = self.get_figure_out_folder('figs')
        
        self.results = None
        self.read_trial_results_from_file()
 
    def get_trial_from_index(self,trial_index):
        
        return trial_data(self.results.iloc[trial_index],self.experiment_settings,self.data_folder)
    
    def read_experiment_settings_from_file(self):

        settings_path = os.path.join(self.data_folder, 'S001', 'session_info', 'settings.json')

        with open(settings_path) as json_data:
            self.experiment_settings = json.load(json_data)
            
    def read_trial_results_from_file(self):

        trial_data_path = os.path.join(self.data_folder, 'S001')
        self.results = pd.read_csv( os.path.join( trial_data_path, 'trial_results.csv' ))
    
class trial_data():
    
    def __init__(self,trial_results_row_in, experiment_settings, subject_data_folder):
        
        self.data_parent_folder = os.path.join(*os.path.split(subject_data_folder)[:-1])
        self.data_folder = subject_data_folder
        
        self.results = trial_results_row_in
        self.experiment_settings = experiment_settings

        # Get raw data
        self.road_vertices = pd.read_csv(os.path.join(self.data_parent_folder, *trial_results_row_in['road_vertices_location_0'].split('/')[1:]))
        self.road_data = pd.read_csv(os.path.join(self.data_parent_folder, *trial_results_row_in['roadTransformMat_location_0'].split('/')[1:]))
        self.car_data = pd.read_csv(os.path.join(self.data_parent_folder,*trial_results_row_in['simplecar_carTransformMatrix_location_0'].split('/')[1:]))

    def get_wheel_angle_data(self):
        # Grabbing the time-stamp and corresponding wheel angle for the trial
        wheel_angles_from_trial = self.car_data[['time','wheelAngle']]
        wheel_angle_array = np.array(wheel_angles_from_trial)
        zeroed_time = []
        
        for t_idx in range(0,len(wheel_angle_array)):
            dif = wheel_angle_array[t_idx, 0] - wheel_angle_array[0,0]
            zeroed_time.append(dif)
        
        wheel_angle_array[:,0] = zeroed_time
        '''
        # plot wheel angle dif over time
        fig, ax = plt.subplots()
        ax.scatter(wheel_angle_array[:,0], wheel_angle_array[:,1])
        plt.xlabel('Time')
        plt.ylabel('Wheel Angle')
        plt.title('Wheel Angle Over Time')
        plt.show()
        '''
        '''
        # Print out max change at a specific time
        max_angle_idx = np.argmax(abs(wheel_angle_array[:,1]))
        
        if wheel_angle_array[max_angle_idx,1]<0:
            max_angle = np.amin(wheel_angle_array[:,1])
        else:
            max_angle = np.amax(wheel_angle_array[:,1])
        
        #print("Max angle deviation = "+ str(max_angle) +" at time-stamp "+ str(wheel_angle_array[max_angle_idx,0]))
        '''
        return wheel_angle_array
    
    def filter_raw_data(self, wheel_angles):
        filtered_time = []
        filtered_wheel_angles = []
        filtered_raw_data = []
        
        for first_term_idx in range(0,len(wheel_angles)-1):
            ang_dif = wheel_angles[first_term_idx+1,1]-wheel_angles[first_term_idx,1]
            time_dif = wheel_angles[first_term_idx+1,0]-wheel_angles[first_term_idx,0]
            if ang_dif/time_dif != 0:
                filtered_time.append(wheel_angles[first_term_idx+1,0])
                filtered_wheel_angles.append(wheel_angles[first_term_idx+1,1])
        
        filtered_raw_data = list(zip(filtered_time,filtered_wheel_angles))
        '''
        # plot
        fig, ax = plt.subplots()
        ax.scatter(filtered_time, filtered_wheel_angles)
        plt.xlabel('Time')
        plt.ylabel('Filtered Wheel Angle')
        plt.title('Filtered Wheel Angles Over Time')
        plt.show()
        '''
        '''
        sampling_rate = len(filtered_wheel_angles)/filtered_time[len(filtered_time)-1]
        
        print("Number of unrepeated wheel angles = "+ str(len(filtered_wheel_angles)))
        print("length of trial = "+ str(filtered_time[len(filtered_time)-1]))
        print("Sampling rate = "+ str(sampling_rate))
        '''
        
        return filtered_raw_data
    


# Analysis

In [14]:
data_parent_folder = 'raw_data'
filenames = ['AGPilot-10-6-22(1)']

subNum = 0
subject_data_folder = os.path.join( data_parent_folder, filenames[subNum])
this_sub_data = subject_data(subject_data_folder)

percentage_removed = []
for idx, row in this_sub_data.results.iterrows():
    a_trial = this_sub_data.get_trial_from_index(idx)
    wheel_angle_over_time = a_trial.get_wheel_angle_data()
    filtered_raw_data = a_trial.filter_raw_data(wheel_angle_over_time)
    perc = (1-len(filtered_raw_data)/len(wheel_angle_over_time)) * 100
    percentage_removed.append(perc)

this_sub_data.results['perc_removed_when_filtered'] = percentage_removed
radius_vs_perc_removed = this_sub_data.results[['turn_radius','perc_removed_when_filtered']]
radius_vs_perc_removed.groupby(['turn_radius']).mean()


,perc_removed_when_filtered
turn_radius,
43.18018,18.440836
50.11290,18.538048
50.89964,20.614486
57.64607,21.780462
58.88199,19.086296
59.50710,22.939694
64.41756,28.188288
65.56192,24.044504


In [15]:
radius_vs_perc_removed.groupby(['turn_radius']).std()

,perc_removed_when_filtered
turn_radius,
43.18018,5.381811
50.11290,5.275465
50.89964,4.382512
57.64607,4.706552
58.88199,6.424506
59.50710,10.206518
64.41756,8.852472
65.56192,5.505165
